<a href="https://colab.research.google.com/github/anajeebb/Data-Mining-Project/blob/main/Data_Mining_Project_High_Value_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining Course Project: High-Value Customer Classification using Logistic Regression

**Student Name:** Abdallah Najeeb
**Student ID:** 202210159
**Course:** Data Mining 307304
**Problem Domain:** Supervised Learning (Classification)

---

###**The Historical Context of Data Mining**
**Data Mining and Machine Learning** have evolved from simple statistical analysis to complex predictive modeling. This transformation, driven by massive data growth and computational power, has moved businesses from simply reporting what happened to predicting what will happen.

###**The Business Problem**
Our project focuses on a critical business challenge for the online retailer: **Customer Retention and Resource Allocation**.
 Not all customers are equally valuable, and marketing resources are limited.

###**The Data Mining Solution (The How)**
The business problem is to accurately identify who these high-value customers are before they make their next purchase.
 We are solving this using **Supervised Learning and Classification technique.**

###**The Chosen Technique: Logistic Regression**
Specifically, we are using **Logistic Regression** to build a predictive model. This model is ideal because it is highly interpretable, allowing the business to understand why a customer is flagged as high-value (e.g., high purchase volume).

###**Expected Business Outcome**
The successful outcome of this project is a model that provides a clear, actionable list of predicted high-value customers.            
**This allows the business to make the right decisions:**  
Allocate premium marketing efforts to the predicted high-value group, maximizing return on investment (ROI) and ensuring strategic growth.

## 1. Dataset Selection & Problem Definition

### - Dataset Selection
The dataset chosen is a **sample** of the **Online Retail** transactional dataset. Reference : https://archive.ics.uci.edu/dataset/352/online+retail

### - Problem Definition: High-Value Customer Classification
The goal is to build a model that can predict whether a customer will be a **High-Value Customer** based on their purchasing behavior. This is a **Supervised Learning Classification** problem.

**Target Variable (Y):** Is_High_Value (Binary: 1 for High-Value, 0 otherwise).
A customer is defined as High-Value if their total spending is in the **top 20%** of all customers.

**Features (X):**
1.  Total_Items: Total number of items purchased.
2.  Total_Invoices: Total number of unique invoices (transactions).
3.  Avg_Unit_Price: Average price of items purchased.

---

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

# Load the dataset from my github
df = pd.read_csv('https://raw.githubusercontent.com/anajeebb/Data-Mining-Project/main/OnlineRetail_Sample.csv')

print(f"Dataset Shape: {df.shape}")

Dataset Shape: (200000, 8)


## 2. Data Cleaning & Preprocessing
### - Data Cleaning
We perform standard cleaning steps: removing rows with missing CustomerID and invalid transactions (non-positive Quantity or UnitPrice).

### - Feature Engineering and Target Creation
The transactional data is aggregated to the customer level to create the features and the binary target variable, Is_High_Value.

In [7]:
# - Data Cleaning
df.dropna(subset=['CustomerID', 'Description'], inplace=True)
df = df[df['Quantity'] > 0]
df = df[df['UnitPrice'] > 0]
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# - Feature Engineering and Target Creation
customer_df = df.groupby('CustomerID').agg(
    Total_Spend=('TotalPrice', 'sum'),
    Total_Items=('Quantity', 'sum'),
    Total_Invoices=('InvoiceNo', 'nunique'),
    Avg_Unit_Price=('UnitPrice', 'mean')
).reset_index()

customer_df.columns = ['CustomerID', 'Total_Spend', 'Total_Items', 'Total_Invoices', 'Avg_Unit_Price']

threshold = customer_df['Total_Spend'].quantile(0.80)
customer_df['Is_High_Value'] = (customer_df['Total_Spend'] >= threshold).astype(int)

print(f"Total Customers: {customer_df.shape[0]}")
print(f"High Value Threshold (80th percentile): {threshold:.2f}")
print(f"High Value Customers (Target=1): {customer_df['Is_High_Value'].sum()} ({customer_df['Is_High_Value'].mean()*100:.2f}%)")

Total Customers: 2658
High Value Threshold (80th percentile): 1258.93
High Value Customers (Target=1): 532 (20.02%)


## 3. Exploratory Data Analysis (EDA)
EDA is performed to understand the distribution of the features and check for multicollinearity.

### - Feature Distributions
The distributions of Total_Items and Total_Invoices are highly skewed, indicating the presence of high-volume buyers (outliers). This skewness is addressed by the subsequent scaling step.



In [8]:
# Select features
X = customer_df[['Total_Items', 'Total_Invoices', 'Avg_Unit_Price']]
Y = customer_df['Is_High_Value']

# Split data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)

# Standard Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("First 5 rows of X_train_scaled:")
print(pd.DataFrame(X_train_scaled, columns=X.columns).head())

First 5 rows of X_train_scaled:
   Total_Items  Total_Invoices  Avg_Unit_Price
0     0.003241       -0.169620       -0.228103
1     1.069568        1.609956       -0.192249
2    -0.175200       -0.423845       -0.122530
3    -0.194805       -0.423845       -0.017091
4    -0.019354        0.084605       -0.145749


## 4. Model Selection & Implementation

### - Model Selection: Logistic Regression
**Logistic Regression** is chosen as it is a fundamental and highly interpretable model for **Binary Classification**, directly aligning with the course material. It predicts the probability of a customer being High-Value using the **Sigmoid Function**.

### - Model Training
The model is trained on the scaled training data.

In [9]:
# Initialize and train the model
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, Y_train)

print("Logistic Regression Model Trained Successfully.")

Logistic Regression Model Trained Successfully.


## 5. Theoretical Understanding of the Model

### - Logistic Regression Theory
Logistic Regression models the probability of a binary outcome using the **Sigmoid Function** (or logistic function), which is represented as: sigma(z) = 1 / (1 + e^(-z)).
The linear combination of features, z = beta_0 + beta_1*x_1 + beta_2*x_2 + ... + beta_n*x_n, is passed through the sigmoid function to produce a probability P(Y=1|X).

### - Model Coefficients
The coefficients (beta_i) represent the change in the log-odds of the target variable for a one-unit increase in the feature. The model's equation is:

Log-Odds = beta_0 + beta_1 * (Total_Items) + beta_2 * (Total_Invoices) + beta_3 * (Avg_Unit_Price)

| Feature | Coefficient | Interpretation (Log-Odds) |
| :--- | :---: | :--- |
| **Total_Items** | **10.3174** | A one-unit increase in the scaled Total_Items dramatically increases the log-odds of being a High-Value Customer. |
| **Total_Invoices** | **2.3067** | A one-unit increase in the scaled Total_Invoices increases the log-odds of being a High-Value Customer. |
| **Avg_Unit_Price** | **0.1199** | This feature has a small positive impact on the prediction. |
| **Intercept (beta_0)** | **-1.2461** | The baseline log-odds when all features are at their mean (scaled to 0). |

The large positive coefficients for Total_Items and Total_Invoices show that **volume** is the primary driver for a customer to be classified as High-Value.

## 6. Evaluation Metrics & Interpretation

The model's performance is evaluated using the key classification metrics: **Accuracy, Precision, Recall, and F1-Score**.


In [10]:
# Make predictions on the test set
Y_pred = model.predict(X_test_scaled)

# Print Classification Report
print(classification_report(Y_test, Y_pred, target_names=['Low Value (0)', 'High Value (1)']))

                precision    recall  f1-score   support

 Low Value (0)       0.92      0.98      0.95       638
High Value (1)       0.90      0.65      0.75       160

      accuracy                           0.91       798
     macro avg       0.91      0.82      0.85       798
  weighted avg       0.91      0.91      0.91       798



### - Classification Report Interpretation
| Metric | Low Value (0) | High Value (1) | Interpretation |
| :--- | :---: | :---: | :--- |
| **Precision** | 0.93 | **0.90** | Of all customers predicted as High-Value, 90% were correct. |
| **Recall** | 0.97 | **0.65** | The model correctly identified 65% of all actual High-Value Customers. |
| **F1-Score** | 0.95 | **0.75** | The harmonic mean of Precision and Recall, indicating a strong balance. |
| **Accuracy** | **0.91** | **0.91** | Overall, 91% of the predictions were correct. |

**Interpretation:** The model has high **Precision** (0.90) for the High-Value class, meaning when it flags a customer as High-Value, it is highly likely to be correct. The **Recall** (0.65) is lower, indicating that 35% of actual High-Value customers were missed (**False Negatives**). Given the business goal of targeting High-Value customers, high Precision is desirable to ensure marketing resources are not wasted on misclassified customers.

##**In Conclusion:**
###**-Technical Summary and Key Findings:**
To conclude the technical analysis, this project successfully applied **Logistic Regression** to the Online Retail dataset, solving a **Binary Classification** problem to predict High-Value Customers. We demonstrated a complete data mining workflow, from data cleaning and Standard Scaling to model training. The model coefficients revealed that **purchase volume**, specifically the number of items and invoices—is the primary statistical driver for a customer to be classified as high-value.

###**-Evaluation and Confidence:**
Moving to the evaluation, the model achieved an overall **Accuracy of 91%.** However, the most critical metric for this business problem is Precision. We achieved a **Precision of 90%** for the High-Value class. This means that when the model flags a customer as high-value, **it is correct 9 out of 10 times.** This high precision provides the business with high confidence in the model's predictions, which is essential for making **resource decisions.**

###**-Business Perspective and Strategic Recommendation:**
Finally, the business perspective is where this project delivers its true value. This model provides **actionable insights** that enable **strategic decision-making.** Our recommendation is to use this predictive tool to prioritize marketing efforts. By targeting the customers predicted as High-Value, the company can ensure their most expensive and personalized campaigns are delivered to the most receptive audience, thereby maximizing their **Return on Investment** and **retaining long-term customer loyalty.** This project moves the business from reactive reporting to **proactive, data-driven strategy.**